In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "utility"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using MorphoMolNotebooks
using StaticArrays
using Rotations
using Distances
using JLD2

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [2]:
#MorphoMolNotebooks.get_low_energy_and_theta_states("../../Data/hpc_out/op/3_6r7m/1/", "6r7m")
#@load "../../Data/hpc_out/rwm_ma_3_6r7m/7_rwm_ma_3_6r7m.jld2" input output
#minimum(get_theta(input, output))

In [7]:
mol_type = "6r7m"
algo = "rwm"
energy = "ma"
n_mol = 3
folder = "../../Data/hpc_out/$energy/$(n_mol)_$(mol_type)/1/"
#folder = "../../Data/hpc_out/$(algo)_$(energy)_$(n_mol)_$(mol_type)/"

id_index = 1
separator = "_"
max = 1
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        max = max < parse(Int, split(file, separator)[id_index]) ? parse(Int, split(file, separator)[id_index]) : max
    end
end

acceptance_rates = Vector{Float64}([])
min_Es = [Inf for _ in 1:max]
thetas = [Inf for _ in 1:max]
evaluation_strings = ["" for _ in 1:max]
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        try
            @load "$folder$file" input output
            # Crude test to exclude runs that remained dispersed
            if maximum(output["Es"]) - minimum(output["Es"]) < 10.0# || input["T"] > 1.5
                println("E_diff = $(maximum(output["Es"]) - minimum(output["Es"]))")
                continue
            end

            id = parse(Int, split(file, separator)[id_index])
            theta = MorphoMolNotebooks.get_theta(input, output)
            string = "$(id): $(input["T"]) |$(output["αs"][end]) | $(minimum(output["Es"])) | $(theta)"

            push!(acceptance_rates, output["αs"][end])
            evaluation_strings[id] = string
            min_Es[id] = minimum(output["Es"])
            thetas[id] = theta
        catch e
            println(e)
            println("Error in $file")
        end
    end
end
println("Minimal energy configuration id: $(argmin(min_Es)) | Average acceptance rate: $(sum(acceptance_rates) / length(acceptance_rates))")
println("Minimal theta configuration id: $(argmin(thetas))")
for elem in evaluation_strings
    println(elem)
end

EOFError()
Error in 97_rwm_ma_3_6r7m.jld2
Minimal energy configuration id: 71 | Average acceptance rate: 0.23887632626362823
Minimal theta configuration id: 42
1: 2.9 | 1.1 |0.19155139 | 13030.830395685785 | 15.967560943242932
2: 2.9 | 1.1 |0.15895095 | 13032.263629457286 | 8.44236917789734
3: 2.9 | 1.1 |0.029697675 | 13029.180464521218 | 13.857639492516634
4: 2.9 | 1.1 |0.1697277 | 13048.672851466637 | 14.622920551974067
5: 2.9 | 1.1 |0.24250509 | 13034.961285006535 | 18.85354612722097
6: 2.9 | 1.1 |0.22821528 | 13032.295736229715 | 21.020617631746486
7: 2.9 | 1.1 |0.120871685 | 13048.18297503137 | 14.693780824925666
8: 2.9 | 1.1 |0.06776636 | 13047.822014961106 | 15.42945241137298
9: 2.9 | 1.1 |0.07079614 | 13033.02396864025 | 19.051691560554193
10: 2.9 | 1.1 |0.29623872 | 13049.664298633132 | 24.360744971693805
11: 2.9 | 1.1 |0.07663321 | 13018.363992695315 | 14.38754003009484
12: 2.9 | 1.1 |0.37458563 | 13050.21231464174 | 20.8433024751557
13: 2.9 | 1.1 |0.11302352 | 13039.14353020

In [6]:
for file in readdir(folder)
    try
        if split(file, ".")[end] == "jld2"
            @load "$folder$file" input output
            mindex = argmin(output["Es"])
            state = output["states"][mindex]
            id = parse(Int, split(file, "_")[id_index])
            radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
            MorphoMolNotebooks.configuration_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(folder)$(id)", input["n_mol"], length(input["template_radii"]))
        end
    catch e
        println(e)
        continue
    end
end

EOFError()


In [8]:
@load "$(folder)/70_$(algo)_$(energy)_$(n_mol)_$(mol_type).jld2" input output

2-element Vector{Symbol}:
 :input
 :output

In [12]:
min_dex = argmin(output["Es"])
state_string = "["
for e in output["states"][min_dex]
    state_string *= "$(e),"
end
state_string *= "]"
state_string

"[33.00573028577495,9.010500374699706,-19.394014994439495,41.94091553913662,36.77396954475643,136.75801336604957,0.025544938985341298,-11.531399621704216,1.9825553928061728,105.15622750806392,30.18769203297849,131.27817550868386,-3.800592120798349,-3.0595641169479544,6.004980837276341,76.64592090269305,28.80512299577225,128.90246903831434,]"

# Print Single Assembly Path

In [4]:
@load "$(folder)/11_$(algo_e)_2_$(mol_type).jld2" input output

LoadError: UndefVarError: `algo_e` not defined

In [ ]:
"$(folder)/11_$(algo_e)_2_$(mol_type).jld2"

"../../Data/hpc_out/2_6r7m/rwm_ma_2_6r7m//11_rwm_ma_2_6r7m.jld2"

In [ ]:
using GeometryBasics

In [ ]:
mindex = argmin(output["Es"])
lower = 500
upper = 100
out_folder = "../../Data/tmp/assembly_path/"
try
    rm(out_folder, recursive=true)
catch 
end
mkdir(out_folder)
radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
for (i,j) in enumerate(maximum([mindex-lower,1]):minimum([mindex+upper, length(output["Es"])]))
    state = output["states"][j]
    MorphoMolNotebooks.configuration_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(out_folder)$(i)", input["n_mol"], length(input["template_radii"]))
end



# Dispersed State Energies

In [ ]:
folder = "assets/input/poly/singles/"
for file in readdir(folder)
    c,r = MorphoMol.Utilities.poly_to_state("$folder$file")
    E = MorphoMol.Energies.solvation_free_energy(
        c, 
        length(r),
        r,
        1.4,
        MorphoMol.Energies.get_prefactors(1.4, 0.3665),
        0.0,
        0.85
        )
    println("$(split(file, "_")[1]) ", 2*E)
end

1ei7 8255.3356279267
2tmv 8130.426867538925
6r7m 8219.578304239454
6sae 8185.217724310747
6sag 8174.157936120334
